In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds

In [2]:
# Load CIFAR-100 dataset
(train_ds, test_ds), info = tfds.load('cifar100', split=['train', 'test'], with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteN7MAT6/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteN7MAT6/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [ ]:
def normalize_img(example):
    image = tf.image.convert_image_dtype(example['image'], tf.float32)
    label = example['label']
    return image, label

In [ ]:
train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# Set batch size
batch_size = 64
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [ ]:
# Resize images to match InceptionV3 input size
input_shape = (299, 299)  # InceptionV3 input size
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, input_shape), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, input_shape), y))

In [ ]:
# Use InceptionV3 as base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build the model on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(100, activation='softmax')  # 100 classes for CIFAR-100
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
782/782 [==============================] - 250s 300ms/step - loss: 2.8493 - accuracy: 0.2971 - val_loss: 1.8500 - val_accuracy: 0.5095
Epoch 2/10
782/782 [==============================] - 225s 287ms/step - loss: 2.1287 - accuracy: 0.4293 - val_loss: 1.6768 - val_accuracy: 0.5411
Epoch 3/10
782/782 [==============================] - 229s 293ms/step - loss: 1.9816 - accuracy: 0.4643 - val_loss: 1.5909 - val_accuracy: 0.5568
Epoch 4/10
782/782 [==============================] - 225s 287ms/step - loss: 1.8961 - accuracy: 0.4822 - val_loss: 1.5757 - val_accuracy: 0.5610
Epoch 5/10
782/782 [==============================] - 225s 287ms/step - loss: 1.8421 - accuracy: 0.4937 - val_loss: 1.5648 - val_accuracy: 0.5575
Epoch 6/10
782/782 [==============================] - 228s 292ms/step - loss: 1.7961 - accuracy: 0.5023 - val_loss: 1.5304 - val_accuracy: 0.5716
Epoch 7/10
782/782 [==============================] - 224s 287ms/step - loss: 1.7534 - accuracy: 0.5131 - val_loss: 1.5053 -

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f'Test accuracy: {test_acc}')

157/157 [==============================] - 38s 241ms/step - loss: 1.4848 - accuracy: 0.5785
Test accuracy: 0.578499972820282
